# Часть 3. SQL/PYTHON

<div style="border:solid black 2px; padding: 20px">

Работа с двумя таблицами:
* products
* order

Необходимо найти следующее:

* Какой средний чек был 13.01?
* Какая доля промо по категории T-shirts?
* Чему равны продажи по категории Trousers?
* Какая маржа в руб и \% по категории Sweatshirts&Hoodies?

</div>

<div style="border:solid black 2px; padding: 20px">

## Загрузка данных.

</div>

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
db_config = {
    'user': '', # имя пользователя
    'pwd': '', # пароль
    'host': '',
    'port': , # порт подключения
    'db': '' # название базы данных
}  

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
) 

In [4]:
engine = create_engine(connection_string)

In [5]:
products_columns = ('''
SELECT TABLE_NAME, 
       COLUMN_NAME,
       DATA_TYPE
FROM information_schema.columns
WHERE table_name = 'products'
;''')

products_cnt = ('''
SELECT COUNT(*) AS cnt_total
FROM test3.products;''')

products_head = ('''
SELECT *
FROM test3.products
LIMIT 5
;''')

print('-'*50)
print('Информация о таблице products')
print('-'*50)
products_columns = pd.read_sql_query(products_columns, con = engine)
products_cnt = pd.read_sql_query(products_cnt, con = engine)
products_head = pd.read_sql_query(products_head, con = engine)
display(products_columns)
display(products_cnt)
display(products_head)

--------------------------------------------------
Информация о таблице products
--------------------------------------------------


,table_name,column_name,data_type
0,products,product_id,integer
1,products,level1,character varying
2,products,level2,character varying
3,products,name,character varying


,cnt_total
0,3573


,product_id,level1,level2,name
0,59272,Antistress and souvenirs,Antistress,"Игрушка ""Пучеглазик"""
1,9096,Antistress and souvenirs,Antistress,Игрушка змейка
2,2506,Antistress and souvenirs,Antistress,Игрушка лизун
3,15017,Antistress and souvenirs,Antistress,Игрушка Тянучка
4,823,Antistress and souvenirs,Antistress,Игрушка-антистресс


In [6]:
order_columns = ('''
SELECT TABLE_NAME, 
       COLUMN_NAME,
       DATA_TYPE
FROM information_schema.columns
WHERE table_name = 'order'
;''')

order_cnt = ('''
SELECT COUNT(*) AS cnt_total
FROM test3.order;''')

order_head = ('''
SELECT *
FROM test3.order
LIMIT 5
;''')

print('-'*50)
print('Информация о таблице order')
print('-'*50)
order_columns = pd.read_sql_query(order_columns, con = engine)
order_cnt = pd.read_sql_query(order_cnt, con = engine)
order_head = pd.read_sql_query(order_head, con = engine)
display(order_columns)
display(order_cnt)
display(order_head)

--------------------------------------------------
Информация о таблице order
--------------------------------------------------


,table_name,column_name,data_type
0,order,order_id,integer
1,order,accepted_at,date
2,order,product_id,integer
3,order,quantity,integer
4,order,regular_price,integer
5,order,price,integer
6,order,cost_price,integer


,cnt_total
0,23582


,order_id,accepted_at,product_id,quantity,regular_price,price,cost_price
0,1516729899,2022-01-10,10163,1,329,329,254
1,1518335746,2022-01-16,10163,1,329,329,229
2,1518354200,2022-01-15,10163,1,329,329,229
3,1516838503,2022-01-11,10163,1,329,329,254
4,1516739758,2022-01-10,10163,1,329,329,254


<div style="border:solid black 2px; padding: 20px">

## Какой средний чек был 13.01?

</div>

In [7]:
avg = ('''
SELECT ROUND(AVG(s.sum)) AS AOV
FROM
(SELECT r.order_id,
       SUM(r.rev) AS sum
FROM       
(SELECT accepted_at,
        order_id,
        product_id,
        (quantity * price) AS rev
FROM test3.order
WHERE accepted_at = '13.01.2022'
GROUP BY accepted_at,
         order_id,
         product_id,
         (quantity * price)) AS r
GROUP BY r.order_id) AS s
;''')

avg = pd.read_sql_query(avg, con = engine)
print(avg)

     aov
0  916.0


<div style="border:solid green 2px; padding: 20px">

Средний чек был 13.01: 916 руб.

</div>

<div style="border:solid black 2px; padding: 20px">

## Какая доля промо по категории T-shirts?

</div>

In [8]:
t_shirts = ('''
SELECT tsh.level1,
       tsh.price_type,
       ROUND(SUM(tsh.revenue) / SUM(tsh.rev_total) * 100) AS perc_revenue,
       ROUND(SUM(tsh.sales_quantity) / SUM(tsh.q_ty_total) * 100) AS perc_sales_q_ty
FROM

(SELECT  ts.level1,
        ts.price_type,
        SUM(ts.revenue) AS revenue,
        SUM(ts.sales_quantity) AS sales_quantity,
        SUM(SUM(ts.revenue)) OVER() AS rev_total,
        SUM(SUM(ts.sales_quantity)) OVER() AS q_ty_total
FROM

(SELECT p.level1,
        o.order_id,
        o.product_id,
        (o.quantity * o.price) AS revenue,
        CASE
        WHEN o.price < o.regular_price THEN 'promo'
        ELSE 'full_price'
        END AS price_type,
        SUM(o.quantity) AS sales_quantity
FROM test3.order AS o
INNER JOIN test3.products AS p ON p.product_id = o.product_id
WHERE p.level1 = 'T-shirts'
GROUP BY p.level1,
         o.order_id,
         o.product_id,
         (o.quantity * o.price),
         CASE
         WHEN o.price < o.regular_price THEN 'promo'
         ELSE 'full_price'
         END) AS ts 
         
GROUP BY ts.level1,
         ts.price_type) AS tsh
         
GROUP BY tsh.level1,
         tsh.price_type
;''')

t_shirts = pd.read_sql_query(t_shirts, con = engine)
display(t_shirts)

,level1,price_type,perc_revenue,perc_sales_q_ty
0,T-shirts,promo,47.0,48.0
1,T-shirts,full_price,53.0,52.0


<div style="border:solid green 2px; padding: 20px">

Доля продаж промо товара (товара на скидке) в категории T-shirts:
* доля продаж в рублях 47%
* доля продаж в штуках 48%

</div>

<div style="border:solid black 2px; padding: 20px">

## Чему равны продажи по категории Trousers?

</div>

In [9]:
trousers = ('''
SELECT tr.level1,
       SUM(tr.revenue) AS revenue,
       SUM(tr.sales_quantity) AS sales_quantity
FROM

(SELECT p.level1,
        o.accepted_at,
        o.order_id,
        o.product_id,
        (o.quantity * o.price) AS revenue,
        SUM(o.quantity) AS sales_quantity
        
FROM test3.order AS o
INNER JOIN test3.products AS p ON p.product_id = o.product_id
WHERE p.level1 = 'Trousers'

GROUP BY p.level1,
         o.accepted_at,
         o.order_id,
         o.product_id,
         (o.quantity * o.price)) AS tr
         
GROUP BY tr.level1
;''')

trousers = pd.read_sql_query(trousers, con = engine)
print(trousers)

     level1  revenue  sales_quantity
0  Trousers    40699           269.0


<div style="border:solid green 2px; padding: 20px">

Продажи по категории Trousers:
* 40699 руб.
* 269 шт.

</div>

<div style="border:solid black 2px; padding: 20px">

## Какая маржа в руб и % по категории Sweatshirts&Hoodies?

</div>

In [10]:
trousers = ('''
SELECT m.level1,
       SUM(m.revenue) - SUM(m.costs) AS margin,
       ROUND((SUM(m.revenue) - SUM(m.costs)) / SUM(SUM(m.revenue)) OVER() * 100, 2) AS margin_perc
FROM

(SELECT p.level1,
        o.accepted_at,
        o.order_id,
        o.product_id,
        (o.quantity * o.price) AS revenue,
        (o.quantity * o.cost_price) AS costs
        
FROM test3.order AS o
INNER JOIN test3.products AS p ON p.product_id = o.product_id
WHERE p.level1 = 'Sweatshirts&Hoodies'

GROUP BY p.level1,
         o.accepted_at,
         o.order_id,
         o.product_id,
         (o.quantity * o.price),
         (o.quantity * o.cost_price)) AS m
         
GROUP BY m.level1
;''')

trousers = pd.read_sql_query(trousers, con = engine)
print(trousers)

                level1  margin  margin_perc
0  Sweatshirts&Hoodies   14949        32.27


<div style="border:solid green 2px; padding: 20px">

Маржа в категории Sweatshirts&Hoodies:
* 14949 руб.
* 32.27\%

</div>